# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapuskasing,49.4169,-82.4331,266.20,79,100,2.57,CA,1701478971
1,1,lubang,13.8584,120.1240,300.76,79,19,11.54,PH,1701479418
2,2,urangan,-25.3000,152.9000,299.01,85,55,8.36,AU,1701479418
3,3,ribeira grande,38.5167,-28.7000,290.80,84,100,8.49,PT,1701479418
4,4,carnarvon,-24.8667,113.6333,297.82,71,9,6.97,AU,1701479002


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.5,
    scale = 0.9,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 297.15 ) & 
                                (city_data_df['Max Temp'] <= 302.15)  &
                                (city_data_df['Humidity'] < 85) &
                                (city_data_df['Cloudiness'] < 60) &
                                (city_data_df['Cloudiness'] > 39) &
                                (city_data_df['Wind Speed'] < 10) 
                                ]

# # Drop any rows with null values
ideal_weather_no_null_df = ideal_weather_df.dropna()

# # Display sample data
ideal_weather_no_null_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,flying fish cove,-10.4217,105.6791,300.97,65,40,4.12,CX,1701479419
70,70,arraial do cabo,-22.9661,-42.0278,298.13,83,40,1.03,BR,1701479027
239,239,sao gabriel da cachoeira,-0.1303,-67.0892,298.39,82,40,0.26,BR,1701479500
243,243,itapirapua,-15.8233,-50.6133,299.95,54,41,1.56,BR,1701479503
291,291,amontada,-3.3617,-39.8317,299.08,80,51,2.68,BR,1701479523
348,348,el porvenir,13.7617,-87.3458,297.78,73,56,2.87,HN,1701479548
389,389,black river,18.0264,-77.8487,298.87,80,49,1.69,JM,1701479565
435,435,brisas de zicatela,15.8369,-97.0419,301.44,57,40,0.00,MX,1701479581
493,493,cockburn town,21.4612,-71.1419,300.13,74,40,2.57,TC,1701479120
551,551,bang saphan,11.2126,99.5117,299.94,80,41,3.45,TH,1701479628


In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = ideal_weather_no_null_df[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,flying fish cove,CX,-10.4217,105.6791,65,
70,arraial do cabo,BR,-22.9661,-42.0278,83,
239,sao gabriel da cachoeira,BR,-0.1303,-67.0892,82,
243,itapirapua,BR,-15.8233,-50.6133,54,
291,amontada,BR,-3.3617,-39.8317,80,
348,el porvenir,HN,13.7617,-87.3458,73,
389,black river,JM,18.0264,-77.8487,80,
435,brisas de zicatela,MX,15.8369,-97.0419,57,
493,cockburn town,TC,21.4612,-71.1419,74,
551,bang saphan,TH,11.2126,99.5117,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "limit": 20,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
flying fish cove - nearest hotel: Christmas Island Lodge
arraial do cabo - nearest hotel: No hotel found
sao gabriel da cachoeira - nearest hotel: Hotel Deus Me Deu
itapirapua - nearest hotel: No hotel found
amontada - nearest hotel: No hotel found
el porvenir - nearest hotel: No hotel found
black river - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
cockburn town - nearest hotel: The Salt Raker Inn
bang saphan - nearest hotel: Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
5,flying fish cove,CX,-10.4217,105.6791,65,Christmas Island Lodge
70,arraial do cabo,BR,-22.9661,-42.0278,83,No hotel found
239,sao gabriel da cachoeira,BR,-0.1303,-67.0892,82,Hotel Deus Me Deu
243,itapirapua,BR,-15.8233,-50.6133,54,No hotel found
291,amontada,BR,-3.3617,-39.8317,80,No hotel found
348,el porvenir,HN,13.7617,-87.3458,73,No hotel found
389,black river,JM,18.0264,-77.8487,80,No hotel found
435,brisas de zicatela,MX,15.8369,-97.0419,57,Casa de Olas
493,cockburn town,TC,21.4612,-71.1419,74,The Salt Raker Inn
551,bang saphan,TH,11.2126,99.5117,80,Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ['Hotel Name', 'Country'],
    size = "Humidity",
    alpha = 0.5,
    scale = 0.9,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)